<a href="https://colab.research.google.com/github/wiz124/chem169-git/blob/main/Li_Harry_RID_016_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Exercise 0
!pip install -q biopython
from Bio import SeqIO
import requests
from io import StringIO

Prot_IDs=['P04406','P10599','P68871','P04637']
def getFasta(ID):
    url = f"https://rest.uniprot.org/uniprotkb/{ID}.fasta"
    response = requests.get(url)
    response.raise_for_status()
    fasta_data = StringIO(response.text)

    for seq_record in SeqIO.parse(fasta_data, "fasta"):
      print(len(seq_record.seq))
      return str(seq_record.seq)


query_dict={}
for id in Prot_IDs:
  query_dict[id]=getFasta(id)

print(query_dict)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 39.2 MB/s eta 0:00:00
335
105
147
393
{'P04406': 'MGKVKVGVNGFGRIGRLVTRAAFNSGKVDIVAINDPFIDLNYMVYMFQYDSTHGKFHGTVKAENGKLVINGNPITIFQERDPSKIKWGDAGAEYVVESTGVFTTMEKAGAHLQGGAKRVIISAPSADAPMFVMGVNHEKYDNSLKIISNASCTTNCLAPLAKVIHDNFGIVEGLMTTVHAITATQKTVDGPSGKLWRDGRGALQNIIPASTGAAKAVGKVIPELNGKLTGMAFRVPTANVSVVDLTCRLEKPAKYDDIKKVVKQASEGPLKGILGYTEHQVVSSDFNSDTHSSTFDAGAGIALNDHFVKLISWYDNEFGYSNRVVDLMAHMASKE', 'P10599': 'MVKQIESKTAFQEALDAAGDKLVVVDFSATWCGPCKMIKPFFHSLSEKYSNVIFLEVDVDDCQDVASECEVKCMPTFQFFKKGQKVGEFSGANKEKLEATINELV', 'P68871': 'MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH', 'P04637': 'MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLG

In [17]:
#Exercise 1
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML


recordlst=[]
for key,value in query_dict.items():
  record=SeqRecord(Seq(value),id=key)
  recordlst.append(record)
with open('query.fasta', "w") as output:
  queryfasta = SeqIO.write(recordlst,output, "fasta")

with open("blast_results.xml") as f:
    blast_records = list(NCBIXML.parse(f))  # one record per query protein

alignment_dict={}
for record in blast_records:
    print(f"\n{record.query}")
    print(record.alignments[:10])
    for alignment in record.alignments[:10]:  # top 10 hits
        hsp = alignment.hsps[0]
        uniprot_id = alignment.hit_def.split("|")[1]  # → "P0A9B2"
        evalue=hsp.expect
        percentidentity=hsp.identities / hsp.align_length
        hit_dict={}
        hit_dict[uniprot_id]={
            'e-value':evalue,
            'percent identity':percentidentity
        }
        query_id=record.query.split(' ')[0]
        alignment_dict[query_id]=hit_dict
print(alignment_dict)



P04406 <unknown description>
[<Bio.Blast.NCBIXML.Alignment object at 0x7bfceba79f40>, <Bio.Blast.NCBIXML.Alignment object at 0x7bfceba7a1e0>]

P10599 <unknown description>
[<Bio.Blast.NCBIXML.Alignment object at 0x7bfceba7aff0>, <Bio.Blast.NCBIXML.Alignment object at 0x7bfceba79af0>, <Bio.Blast.NCBIXML.Alignment object at 0x7bfceba7aea0>]

P68871 <unknown description>
[]

P04637 <unknown description>
[]
{'P04406': {'P0A9B6': {'e-value': 1.24916e-82, 'percent identity': 0.375}}, 'P10599': {'P77395': {'e-value': 2.36368e-06, 'percent identity': 0.25274725274725274}}}
